# Set configuration

You need to login to a server and fill the following variables:

In [ ]:
from IPython.core.display import display
import ipywidgets as widgets

session_widget = widgets.Text(placeholder='PHPSESSID', description='Session ID')
server_widget = widgets.Text(value='s1.railnation.de', description='Server URL')
lang_widget = widgets.IntText(value=0, description='Language ID')

display(session_widget, server_widget, lang_widget)

In [ ]:
session_id = session_widget.value
server_url = server_widget.value
lang_code = lang_widget.value


In [ ]:
# get user ID
from server import ServerCaller

api = ServerCaller(server_url, session_id)
server = api.call('ServerInfoInterface', 'getInfo', None, short_call=1092)
user_id = server['Infos']['activeUser']
print(f'Logged in to {server["Body"]["worldName"]}')

In [ ]:
# get tracks
tracks = api.call('RailInterface', 'getForUser', data=[user_id], short_call=1236)
track_edges = [(track['FromId'], track['ToId']) for track in tracks['Body']]

import networkx as nx

tracks_graph = nx.Graph()
tracks_graph.add_edges_from(track_edges)
print(f'Layed {len(track_edges)} tracks')

In [ ]:
# connected towns
from city_names import city_names

towns = api.call('LocationInterface', 'getAllTowns', short_call=1236)

# TODO fix mapping towns -> names
#for i, town in enumerate(towns['Body']):
#    town['Name'] = city_names[str(i+1)][str(lang_code)]

towns = [town for town in towns['Body'] if town['Id'] in tracks_graph.nodes]
print(f'Connected to {len(towns)} towns')


In [ ]:
# connected factories
factories = api.call('LocationInterface', 'getAllFactories', short_call=1236)

factories = [factory for factory in factories['Body'] if factory['Id'] in tracks_graph.nodes]

factory_details = list(api.bulk_call('LocationInterface', 'getFactoryDetails', [factory['Id'] for factory in factories]))
factory_details

In [ ]:
# tabular factory details
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

data = [{
        'Id': factory['Parameters'][0],
        'Level': factory['Body']['Level'],
        'WaitTime': factory['Body']['StoragesInfo']['WaitTime']['final'],
        'WorkLoad': factory['Body']['WorkloadInfo']['Workload'],
        'GrowthPoints': factory['Body']['WorkloadInfo']['GrowthPoints'],
        'GrowthTrend': factory['Body']['WorkloadInfo']['Trend'],
     } for factory in factory_details]

df = pd.DataFrame(data)
df


In [ ]:
# train spotters
spotters = api.call('TrainSpotterInterface', 'getWaitingAndCollected', short_call=1236)
print(f'Found {len(spotters["Body"]["Waiting"]["TrainSpotter"])} train spotters')

collected_spotters = list(api.bulk_call('TrainSpotterInterface', 'collect', spotters['Body']['Waiting']['TrainSpotter']))
print(f'Collected {len(collected_spotters)} train spotters')
